<a href="https://colab.research.google.com/github/shashavali-d/Challenges/blob/master/6SenseWorkItem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purchase Prediction using Tensorflow Estimator API**

In [18]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
#do not show warnings
import warnings 
warnings.filterwarnings("ignore")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!unzip /content/drive/My\ Drive/6takehome.zip 

Archive:  /content/drive/My Drive/6takehome.zip
   creating: takehome/
  inflating: takehome/readme.txt     
  inflating: takehome/test.tsv       
  inflating: takehome/training.tsv   


In [0]:
#Read the train/test dataset in to Pandas datframe and assuming dataset path as current working directory

train_columns = ['user_id', 'activity_date', 'activity_type']

dftrain = pd.read_csv('./takehome/training.tsv', names = train_columns, sep = '\t')

dftest = pd.read_csv('./takehome/test.tsv', names = train_columns, sep = '\t')

 **Remove unnecessary columns 'CustomerSupport' here**

>

In [0]:
df = dftrain.loc[dftrain['activity_type'] != 'CustomerSupport']

In [0]:
activity_type_map = {
   'EmailOpen': 1,
   'EmailClickthrough': 2,
   'WebVisit': 3, 
   'PageView': 4, 
   'FormSubmit' : 5,
   'Purchase': 6  
}

In [0]:
df['activity_seq'] = df['activity_type'].apply(lambda x: activity_type_map[x])

In [10]:
#df_group = df.groupby(['user_id', 'activity_date', 'activity_type']).sum().reset_index()
#df = df.drop(['activity_date', 'activity_type'], axis = 1)
df.head(5)

,user_id,activity_date,activity_type,activity_seq
0,00002acbe042d69,2013-07-11,EmailOpen,1
1,00002acbe042d69,2013-07-11,FormSubmit,5
2,00002acbe042d69,2013-07-15,EmailOpen,1
3,00002acbe042d69,2013-07-17,EmailOpen,1
4,00002acbe042d69,2013-07-18,EmailOpen,1


In [11]:
grouped_df = df.groupby(['user_id'])
user_act_list = []
for i, j in grouped_df:
  temp_list = grouped_df.get_group(i)['activity_seq'].astype('int64').to_list()
  temp_dict = {}
  temp_dict['user_id'] = i
  temp_dict['len'] = len(temp_list)
  temp_dict['activity_sequence'] = temp_list
  temp_dict['label'] = 1 if 6 in temp_list else 0
  user_act_list.append(temp_dict)

train_df = pd.DataFrame(user_act_list)
len(train_df)

341568

In [14]:
train_df.tail(4)

,user_id,len,activity_sequence,label
341564,ffffa4cc0190881,1,[1],0
341565,ffffa7beb541707,3,"[1, 5, 1]",0
341566,ffffecdd8dc6174,14,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
341567,fffff031760c3f5,5,"[1, 1, 1, 1, 1]",0


In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341568 entries, 0 to 341567
Data columns (total 4 columns):
user_id              341568 non-null object
len                  341568 non-null int64
activity_sequence    341568 non-null object
label                341568 non-null int64
dtypes: int64(2), object(2)
memory usage: 10.4+ MB


In [13]:
dftest['activity_seq'] = dftest['activity_type'].apply(lambda x: activity_type_map[x])
grouped_testdf = dftest.groupby(['user_id'])
testuser_act_list = []
for i, j in grouped_testdf:
  temp_list = grouped_testdf.get_group(i)['activity_seq'].astype('int64').to_list()
  temp_dict = {}
  temp_dict['user_id'] = i
  temp_dict['len'] = len(temp_list)
  temp_dict['activity_sequence'] = temp_list
  testuser_act_list.append(temp_dict)

test_df = pd.DataFrame(testuser_act_list)
len(test_df)

132810

In [0]:
labels = train_df.pop('label')

In [0]:
# Divide the features in to numeric and categorical
NUMERICAL_COLUMNS = ['len', 'activity_sequence']

In [0]:
feature_columns = []
for header in NUMERICAL_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(header))

In [0]:
def train_input_fn(dataset, lab):
    return  tf.estimator.inputs.pandas_input_fn(dataset, queue_capacity = 1000,
                                               y = lab, batch_size = 64, shuffle = True, num_epochs = 3)

In [25]:
model = tf.estimator.DNNClassifier(feature_columns=feature_columns, 
                                       hidden_units=[10, 20, 40, 80], 
                                       model_dir = './',
                                       optimizer=lambda: tf.train.AdamOptimizer(
                                           learning_rate=tf.train.exponential_decay(
                                               learning_rate=0.1,
                                               global_step=tf.train.get_global_step(),
                                               decay_steps=10000,
                                               decay_rate=0.96)))
trained_model = model.train(input_fn=train_input_fn(train_df, labels), steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f674b51ec18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using 

InternalError: ignored